In [24]:
#Load libraries
# Note 1 - This code is to import all of the libraries needed for the coding of a CNN, 
import os #interact with operating system
import numpy as np #numpy arrays
import torch #PyTorch tensors
import glob #useful in list of files
import torch.nn as nn #import main neural network libraries
from torchvision.transforms import transforms #transform input images of crops and weeds
from torch.utils.data import DataLoader #Load the data from local computer
from torch.optim import Adam #Optimization algorithms, loss function
from torch.autograd import Variable #calculate gradients for backward propogation
import torchvision #Used to process images of crops and weeds
import pathlib #easier to read files within folder directory

In [25]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
print(device)

cpu


In [27]:
#Transforms
# Note2 - This code transforms all of the images in the local computer given to the algorithm from
# whatever pixels they are, to 150x150 pixels to make the images easier to classify
# makes the RGB 0-255 become 0-1 values
#normalize function is used to bring pixel value from -1 to 1, helps CNN learn faster and better
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [30]:
#Dataloader
#Note 3 - This part of the code tells the computer where the files are for training and testing path
# Then, the data gets transformed in smaller batch sizes with train in batches of 64 and test in 32

#Path for training and testing directory
train_path='C:/Users/mojav/OneDrive - Saputo Inc/Self/Pranay/Python/Crop Detection Project/Clean Data/train_data'
#'/home/user/Desktop/pytorch_projects/scene_detection/seg_train/seg_train'
test_path='C:/Users/mojav/OneDrive - Saputo Inc/Self/Pranay/Python/Crop Detection Project/Clean Data/test_data'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [31]:
#categories, get the 2 output classes from the directory names
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [32]:
print(classes)

['crop', 'weed']


In [33]:
#CNN Network
#Note 4 - This part of the code is where all of the layers of the CNN is made in order to have a good
# accuracy for showing whether the image is a weed or a crop, there are 3 layers of 12, 20 , 32 channels
#I also used relu function for non-linearity 

class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forward function
        #Note 5 - This subsection of the code is creating a forward function for CNN.
        #this is the outputs for all of the layers made above, 
        #this is in matrix form of 256,32,75,75 this makes the outputs of the inputs above
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [34]:
model=ConvNet(num_classes=2).to(device)

In [35]:
#Optmizer and loss function
#Note 6 - This is the data optimizer with weights and loss functions in order to create a stronger model
# the loss function accurately shows how well the algorithm models the dataset. 
#CrossEntropyLoss is often used for CNN image detection
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [36]:
# Notes 7 - There are 10 epochs to see how well the data runs in some different weights, 
# the rule of thumb is at least 3x the number of columns, and this is a bit over because the accuracy 
#was becoming better with more and more epochs
num_epochs=10

In [37]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpeg'))
test_count=len(glob.glob(test_path+'/**/*.jpeg'))

In [38]:
print(train_count,test_count)

908 380


In [39]:
#Model training and saving best model
#Note 8 -  In this part we are putting everything together to train the model and test it
# We are doing a 'for' loop for 10 epochs to carryout loading of images, forward function, loss 
#calculation and backward propogation
# we are also calculating training and testing accuracy for each epoch. The model with best 
# accuracy is saved as 'best_checkpoint.model' which we will use later to predict new images

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       


Epoch: 0 Train Loss: tensor(4.6150) Train Accuracy: 0.7830396475770925 Test Accuracy: 0.49473684210526314
Epoch: 1 Train Loss: tensor(1.5577) Train Accuracy: 0.8678414096916299 Test Accuracy: 0.5842105263157895
Epoch: 2 Train Loss: tensor(0.5809) Train Accuracy: 0.9504405286343612 Test Accuracy: 0.7894736842105263
Epoch: 3 Train Loss: tensor(0.3259) Train Accuracy: 0.9548458149779736 Test Accuracy: 0.9105263157894737
Epoch: 4 Train Loss: tensor(0.4288) Train Accuracy: 0.9625550660792952 Test Accuracy: 0.8947368421052632
Epoch: 5 Train Loss: tensor(1.7144) Train Accuracy: 0.8942731277533039 Test Accuracy: 0.8947368421052632
Epoch: 6 Train Loss: tensor(0.6590) Train Accuracy: 0.9570484581497798 Test Accuracy: 0.8368421052631579
Epoch: 7 Train Loss: tensor(0.2394) Train Accuracy: 0.9790748898678414 Test Accuracy: 0.9026315789473685
Epoch: 8 Train Loss: tensor(0.3061) Train Accuracy: 0.9834801762114538 Test Accuracy: 0.9210526315789473
Epoch: 9 Train Loss: tensor(0.3475) Train Accuracy: 0.